In [0]:
import numpy as np
import pandas as pd 
import requests 
from bs4 import BeautifulSoup 


# **Introduction**: **Business** **Problem**

In this project I would like to find an optimal location for a dessert shop in Paris.

The idea is to find a place in the city center, near some clothes shop or landmark in order to have more tourists. It's important that this shop is far from other similar shops.

I'll use web scraping in order to get information about possible neighborhoods and Foresquare in orger to get information about shops.



# **Data**

The most important criteria is the number of existing dessert shops in the neighborhood.
I'll divide Paris according to its arrondissements to define neighborhoods, centered around city center; for each arrondissement I will find the dessert shops in order to calculate the best location for the new shop.

Each arrondissement will be approximated using Google Maps API geocoding, every activity its type and location using Foursquare API.
The center of Paris will be approximated using Google Maps API too.




In [2]:
#Web scraping
URL = "https://en.wikipedia.org/wiki/Arrondissements_of_Paris#Arrondissements"
page = requests.get(URL) 
print(page)   
soup = BeautifulSoup(page.content, 'html.parser') 


<Response [200]>


Read the table


In [0]:
#get the value from the table
arrondissement  = [];
name=[];
i=0
for row in soup.find('table',class_="wikitable").find_all('tr'):
    
    if i!=0:
      x=row.find_all('td')
      
      arrondissement.append(x[0].text)
      name.append(x[1].text)
    i=i+1 


In [0]:
# define the dataframe 
#arrondissement_number=list(range(1,21))
arrondissement_number = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th','10th',
                '11th', '12th', '13th', '14th', '15th', '16th', '17th', '18th', '19th', '20th']
neighbors = pd.DataFrame({'Arrondissement':arrondissement, 'Name':name,'Arrondissement_number':arrondissement_number})
#clean data
#neighbors = neighbors.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'(+w) R',  ' ', regex=True)
#neighbors =neighbors.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r' / ',  ',', regex=True)

In [21]:
neighbors.head(26)

,Arrondissement,Name,Arrondissement_number
0,1st (Ier) R,Louvre,1st
1,2nd (IIe) R,Bourse,2nd
2,3rd (IIIe) R,Temple,3rd
3,4th (IVe) R,Hôtel-de-Ville,4th
4,5th (Ve) L,Panthéon,5th
5,6th (VIe) L,Luxembourg,6th
6,7th (VIIe) L,Palais-Bourbon,7th
7,8th (VIIIe) R,Élysée,8th
8,9th (IXe) R,Opéra,9th
9,10th (Xe) R,Entrepôt,10th


In [6]:
neighbors.shape

(20, 3)

## **Get** **Cordinates**

In [7]:
import sys
!{sys.executable} -m pip install geocoder

In [0]:
import geocoder 
import requests

In [33]:
from geopy.geocoders import Nominatim
lat_lng_coords = None
latitude=[];
longitude=[];
for code in range(0, len(neighbors['Arrondissement'])):      
    #g = geocoder.arcgis('{}, Paris, France'.format(code))
    #lat_lng_coords = g.latlng
    #latitude.append(lat_lng_coords[0])
    #longitude.append(lat_lng_coords[1])

    address = (str(neighbors['Arrondissement_number'][code]) + ' Arrondissement')
    geolocator = Nominatim(user_agent="arrond_explorer")
    location = geolocator.geocode(address)
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    print('{}, {}, {},'.format(address,latitude,longitude))

1st Arrondissement, [48.8621576], [2.337035643537315],
2nd Arrondissement, [48.8621576, 48.8676844], [2.337035643537315, 2.343126295040478],
3rd Arrondissement, [48.8621576, 48.8676844, 48.86268345], [2.337035643537315, 2.343126295040478, 2.3586851044879937],
4th Arrondissement, [48.8621576, 48.8676844, 48.86268345, 48.85415595], [2.337035643537315, 2.343126295040478, 2.3586851044879937, 2.3567892289437014],
5th Arrondissement, [48.8621576, 48.8676844, 48.86268345, 48.85415595, 48.8454189], [2.337035643537315, 2.343126295040478, 2.3586851044879937, 2.3567892289437014, 2.3525815263156975],
6th Arrondissement, [48.8621576, 48.8676844, 48.86268345, 48.85415595, 48.8454189, 48.8493919], [2.337035643537315, 2.343126295040478, 2.3586851044879937, 2.3567892289437014, 2.3525815263156975, 2.3322597335758593],
7th Arrondissement, [48.8621576, 48.8676844, 48.86268345, 48.85415595, 48.8454189, 48.8493919, 48.85491365], [2.337035643537315, 2.343126295040478, 2.3586851044879937, 2.3567892289437014, 

In [34]:
neighbors['Latitude']=latitude
neighbors['Longitude']=longitude
neighbors.head()

,Arrondissement,Name,Arrondissement_number,Latitude,Longitude
0,1st (Ier) R,Louvre,1st,48.862158,2.337036
1,2nd (IIe) R,Bourse,2nd,48.867684,2.343126
2,3rd (IIIe) R,Temple,3rd,48.862683,2.358685
3,4th (IVe) R,Hôtel-de-Ville,4th,48.854156,2.356789
4,5th (Ve) L,Panthéon,5th,48.845419,2.352582


In [35]:
neighbors.head(20)

,Arrondissement,Name,Arrondissement_number,Latitude,Longitude
0,1st (Ier) R,Louvre,1st,48.862158,2.337036
1,2nd (IIe) R,Bourse,2nd,48.867684,2.343126
2,3rd (IIIe) R,Temple,3rd,48.862683,2.358685
3,4th (IVe) R,Hôtel-de-Ville,4th,48.854156,2.356789
4,5th (Ve) L,Panthéon,5th,48.845419,2.352582
5,6th (VIe) L,Luxembourg,6th,48.849392,2.332260
6,7th (VIIe) L,Palais-Bourbon,7th,48.854914,2.312860
7,8th (VIIIe) R,Élysée,8th,48.877480,2.317650
8,9th (IXe) R,Opéra,9th,48.877070,2.337921
9,10th (Xe) R,Entrepôt,10th,48.876008,2.360445


# Dataset exploration

In [34]:
!{sys.executable} -m pip install folium

In [0]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [26]:
address = ('Paris, France')
geolocator = Nominatim(user_agent="Paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('{''}, {}, {},'.format(address,latitude,longitude))


Paris, France, 48.8566969, 2.3514616,


In [36]:
## Create a map of Paris using latitude and longitude values 
map_paris = folium.Map(location=[48.8566969, 2.3514616], zoom_start=15)
folium.Marker([48.8566969, 2.3514616], popup='Paris').add_to(map_paris)
# add markers to map
for i in range(0,len(neighbors)):
    label = '{}'.format(neighbors['Name'][i])
    label = folium.Popup(label, parse_html=True)
    print(i)
    folium.CircleMarker(
        [neighbors['Latitude'][i], neighbors['Longitude'][i]],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html = False
        ).add_to(map_paris)  
    
map_paris



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


## Foursquare
let's use Foursquare API to get info on each activity in each neighborhood.

I'm interested in acitivity that sell dessert, cupcake, chocolate...


In [38]:
VERSION = '20180605' # Foursquare API version

neighborhood_name = neighbors.loc[0, 'Arrondissement'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
        # https://api.foursquare.com/v2/venues/search?ll=40.7,-74&client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=YYYYMMDD
# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, XD0MCDPMOCYAVDK3K20GIK33WTNUG0RNGYVRWFEOQTKDQCVI
#     CLIENT_SECRET, ICMDJNT2ESQQ1UID1CET3ZX43ORMPCQQHP2KA3KQWOHG4Z55
#     VERSION, 
#     48.8566, 2.3522
#     radius, 
#     LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore?ll=48.8566, 2.3522&client_id=XD0MCDPMOCYAVDK3K20GIK33WTNUG0RNGYVRWFEOQTKDQCVI&client_secret=ICMDJNT2ESQQ1UID1CET3ZX43ORMPCQQHP2KA3KQWOHG4Z55&v=20180605&radius=500&limit=100'

results = requests.get(url).json()
results

Latitude and longitude values of 1st (Ier) R are 48.8621576, 2.337035643537315.


{'meta': {'code': 200, 'requestId': '5e8772571e152c001b7bdd23'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bf41231e5eba59334341f90-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d164941735',
         'name': 'Plaza',
         'pluralName': 'Plazas',
         'primary': True,
         'shortName': 'Plaza'}],
       'id': '4bf41231e5eba59334341f90',
       'location': {'address': "Place de l'Hôtel de Ville",
        'cc': 'FR',
        'city': 'Paris',
        'country': 'France',
        'distance': 68,
        'formattedAddress': ["Place de l'Hôtel de Ville",
         '75004 Paris',
         'France'],
        'lat': 48.85692475726913,
        'lng': 2.3514118156673676,
        'po

In [0]:
# I explore the category in order to find the best places

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Place de l'Hôtel de Ville – Esplanade de la Li...,Plaza,48.856925,2.351412
1,Parc Rives de Seine,Park,48.855510,2.351419
2,Maison Aleph,Pastry Shop,48.857348,2.354873
3,Berges de Seine – Rive droite,Pedestrian Plaza,48.855131,2.352289
4,Galerie Azzedine Alaïa,Art Gallery,48.857545,2.355217


In [40]:
nearby_venues.groupby('name').count()

,categories,lat,lng
name,,,
Amorino,1,1,1
Archive 18-20,1,1,1
Au Bougnat,1,1,1
Au Vieux Paris d'Arcole,1,1,1
Autour du Saumon,1,1,1
...,...,...,...
Une Glace à Paris,1,1,1
Vingt Vins d'Art,1,1,1
Who's,1,1,1


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Explore all neighborhoods in Paris

In [0]:
def getNearbyVenues(names, latitudes, longitudes):
    VERSION = '20180605' # Foursquare API version
    radius = 2000 # define radius
    LIMIT = 300 # limit of number of venues returned by Foursquare API
    CLIENT_ID='XD0MCDPMOCYAVDK3K20GIK33WTNUG0RNGYVRWFEOQTKDQCVI'
    CLIENT_SECRET='ICMDJNT2ESQQ1UID1CET3ZX43ORMPCQQHP2KA3KQWOHG4Z55'
    venues_list=[]
    
   
    

    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, lat, lng)

        # https://api.foursquare.com/v2/venues/search?ll=40.7,-74&client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=YYYYMMDD
# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, XD0MCDPMOCYAVDK3K20GIK33WTNUG0RNGYVRWFEOQTKDQCVI
#     CLIENT_SECRET, ICMDJNT2ESQQ1UID1CET3ZX43ORMPCQQHP2KA3KQWOHG4Z55
#     VERSION, 
#     48.8566, 2.3522
#     radius, 
#     LIMIT)



        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
             CLIENT_ID, 
             CLIENT_SECRET, 
             VERSION, 
             lat, 
             lng, 
             radius, 
             LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
        

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Arrondissement', 
                   'Arrondissement Latitude', 
                  'Arrondissement Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                 'Venue Category']
    
    return(nearby_venues)


            
   

Get venues for all Neighborhoods

In [113]:
paris_venues = getNearbyVenues(names=neighbors['Arrondissement_number'],
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

1st 48.8621576 2.337035643537315
2nd 48.8676844 2.343126295040478
3rd 48.86268345 2.3586851044879937
4th 48.85415595 2.3567892289437014
5th 48.8454189 2.3525815263156975
6th 48.8493919 2.3322597335758593
7th 48.85491365 2.312859739726366
8th 48.8774799 2.31765
9th 48.87706995 2.3379212854897715
10th 48.876008049999996 2.360445036409199
11th 48.860071149999996 2.3781433620616133
12th 48.83520025 2.445135854630304
13th 48.82985295 2.3630259501036406
14th 48.8295667 2.3239624642685364
15th 48.841430200000005 2.296164933019397
16th 48.85715055 2.2567311894375015
17th 48.88743895 2.3065228678413767
18th 48.89212585 2.3481775453229785
19th 48.88721865 2.3831026295673827
20th 48.8618271 2.4010730598176093


In [114]:
paris_venues

,Arrondissement,Arrondissement Latitude,Arrondissement Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1st,48.862158,2.337036,Musée du Louvre,48.860847,2.336440,Art Museum
1,1st,48.862158,2.337036,Palais Royal,48.863236,2.337127,Historic Site
2,1st,48.862158,2.337036,Comédie-Française,48.863088,2.336612,Theater
3,1st,48.862158,2.337036,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza
4,1st,48.862158,2.337036,Cour Napoléon,48.861172,2.335088,Plaza
...,...,...,...,...,...,...,...
1977,20th,48.861827,2.401073,Paris New York,48.865843,2.377702,Burger Joint
1978,20th,48.861827,2.401073,La Commune,48.869821,2.379622,Cocktail Bar
1979,20th,48.861827,2.401073,Le Saint Sauveur,48.866442,2.385305,Rock Club
1980,20th,48.861827,2.401073,Le Réfectoire,48.873382,2.401105,Food Truck


In [115]:
paris_venues.groupby('Venue').count()

,Arrondissement,Arrondissement Latitude,Arrondissement Longitude,Venue Latitude,Venue Longitude,Venue Category
Venue,,,,,,
& Other Stories,1,1,1,1,1,1
0fr.,3,3,3,3,3,3
142 Crêperie Contemporaine,1,1,1,1,1,1
5 Pailles,1,1,1,1,1,1
52 Faubourg Saint-Denis,3,3,3,3,3,3
...,...,...,...,...,...,...
Élysée Montmartre,1,1,1,1,1,1
Élysées Hong-Kong,1,1,1,1,1,1
Éléphant Paname,3,3,3,3,3,3


In [0]:
#There are some duplicates of venues, so we will get rid of the duplicates and keep one copy
paris_venues = paris_venues.drop_duplicates(subset=['Venue'], keep = 'first')

In [0]:
paris_venues = paris_venues.reset_index(drop=True)

In [118]:
# check the size of the resulting dataframe

print(paris_venues.shape)
paris_venues

(1264, 7)


,Arrondissement,Arrondissement Latitude,Arrondissement Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1st,48.862158,2.337036,Musée du Louvre,48.860847,2.336440,Art Museum
1,1st,48.862158,2.337036,Palais Royal,48.863236,2.337127,Historic Site
2,1st,48.862158,2.337036,Comédie-Française,48.863088,2.336612,Theater
3,1st,48.862158,2.337036,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza
4,1st,48.862158,2.337036,Cour Napoléon,48.861172,2.335088,Plaza
...,...,...,...,...,...,...,...
1259,20th,48.861827,2.401073,Ragazzi,48.850873,2.392455,Italian Restaurant
1260,20th,48.861827,2.401073,Piscine Georges Vallerey | Piscine des Tourelles,48.875480,2.406684,Pool
1261,20th,48.861827,2.401073,Le Saint Sauveur,48.866442,2.385305,Rock Club
1262,20th,48.861827,2.401073,Le Réfectoire,48.873382,2.401105,Food Truck


In [119]:
# check how many venues were returned for each arrondissement

paris_venues.groupby('Arrondissement').count()

,Arrondissement Latitude,Arrondissement Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Arrondissement,,,,,,
10th,29,29,29,29,29,29
11th,52,52,52,52,52,52
12th,78,78,78,78,78,78
13th,85,85,85,85,85,85
14th,79,79,79,79,79,79
15th,62,62,62,62,62,62
16th,90,90,90,90,90,90
17th,75,75,75,75,75,75
18th,58,58,58,58,58,58


In [120]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(paris_venues['Venue Category'].unique())))

There are 206 uniques categories.


In [121]:
paris_venues['Venue Category'].unique()

array(['Art Museum', 'Historic Site', 'Theater', 'Pedestrian Plaza',
       'Plaza', 'Exhibit', 'Garden', 'Udon Restaurant',
       'French Restaurant', 'Bridge', 'Church', 'Bakery',
       'Cosmetics Shop', 'Fountain', 'Cocktail Bar', 'Italian Restaurant',
       "Women's Store", 'Israeli Restaurant', 'Hotel', 'Pastry Shop',
       'Spa', 'Wine Bar', 'Toy / Game Store', 'Art Gallery',
       'Dessert Shop', 'Cycle Studio', 'Souvlaki Shop', 'Bookstore',
       'Pizza Place', 'Restaurant', "Men's Store", 'Sandwich Place',
       'Hotel Bar', 'Jewelry Store', 'Beer Bar', 'Ice Cream Shop',
       'Seafood Restaurant', 'Boutique', 'Clothing Store',
       'Breton Restaurant', 'Lebanese Restaurant', 'Concert Hall',
       'Gym / Fitness Center', 'Electronics Store', 'Souvenir Shop',
       'Indie Movie Theater', 'Opera House', 'Japanese Restaurant',
       'Creperie', 'Corsican Restaurant', 'Coffee Shop',
       'Szechuan Restaurant', 'Vegetarian / Vegan Restaurant', 'Bistro',
       'Roof 

In [122]:
print('List of all dessert shops')
print('-----------------------')

for i in range(0, len(paris_venues))[:70]:
  if paris_venues['Venue Category'][i]in{'Pastry Shop','Café','Dessert Shop','Cupcake Shop','Chocolate Shop','Creperie','Candy Store'}:
    print(paris_venues['Venue'][i])
print('...')


List of all dessert shops
-----------------------
Pierre Hermé
Ladurée
...


In [0]:
#underline interesting activities
is_dessert=[];
for i in range(0, len(paris_venues)):
  #print(paris_venues['Venue Category'][i])
  if paris_venues['Venue Category'][i]in{'Pastry Shop','Café','Dessert Shop','Cupcake Shop','Chocolate Shop','Creperie','Candy Store'}:
    is_dessert.append(True)
  else:
    #print(paris_venues['Venue Category'][i])
    is_dessert.append(False)


In [129]:
paris_venues['IsDessert']=is_dessert

paris_venues.head()

,Arrondissement,Arrondissement Latitude,Arrondissement Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,IsDessert
0,1st,48.862158,2.337036,Musée du Louvre,48.860847,2.336440,Art Museum,False
1,1st,48.862158,2.337036,Palais Royal,48.863236,2.337127,Historic Site,False
2,1st,48.862158,2.337036,Comédie-Française,48.863088,2.336612,Theater,False
3,1st,48.862158,2.337036,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza,False
4,1st,48.862158,2.337036,Cour Napoléon,48.861172,2.335088,Plaza,False


I am looking for Café

In [130]:
paris_venues.shape

(1264, 8)

In [0]:
from geopy.geocoders import Nominatim

In [132]:
address = ('Paris, France')
geolocator = Nominatim(user_agent="Paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('{''}, {}, {},'.format(address,latitude,longitude))

Paris, France, 48.8566969, 2.3514616,


In [133]:
## Create a map of Paris using latitude and longitude values 
map_paris = folium.Map(location=[48.8566969, 2.3514616], zoom_start=15)
folium.Marker([48.8566969, 2.3514616], popup='Paris').add_to(map_paris)
# add markers to map
for i in range(0,len(paris_venues)):
    label = '{}'.format(paris_venues['Venue'][i])
    label = folium.Popup(label, parse_html=True)
    is_Dessert = paris_venues['IsDessert'][i]
    color = 'red' if is_Dessert else 'blue'
    
    folium.CircleMarker(
        [paris_venues['Venue Latitude'][i], paris_venues['Venue Longitude'][i]],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=1).add_to(map_paris)  
    
map_paris



In [0]:
#paris_venues1=paris_venues
paris_venues=paris_venues[paris_venues['IsDessert']==True]
paris_venues = paris_venues.reset_index(drop=True)

In [176]:
# one hot encoding
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Arrondissement'] = paris_venues['Arrondissement'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,Arrondissement,Café,Candy Store,Chocolate Shop,Creperie,Cupcake Shop,Dessert Shop,Pastry Shop
0,1st,0,0,0,0,0,0,1
1,1st,0,0,0,0,0,1,0
2,2nd,0,0,0,1,0,0,0
3,2nd,0,0,1,0,0,0,0
4,3rd,0,0,0,1,0,0,0


In [177]:
# examine the new dataframe size.

paris_onehot.shape

(41, 8)

In [178]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

paris_grouped = paris_onehot.groupby('Arrondissement').mean().reset_index()
paris_grouped

,Arrondissement,Café,Candy Store,Chocolate Shop,Creperie,Cupcake Shop,Dessert Shop,Pastry Shop
0,12th,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
1,13th,0.333333,0.0,0.000000,0.333333,0.000000,0.333333,0.00
2,14th,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.00
3,15th,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.00
4,16th,0.666667,0.0,0.000000,0.000000,0.000000,0.333333,0.00
5,17th,0.000000,0.0,0.000000,0.500000,0.000000,0.000000,0.50
6,18th,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
7,19th,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
8,1st,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.50
9,20th,0.000000,0.0,0.500000,0.500000,0.000000,0.000000,0.00


In [179]:
# confirm the new size

paris_grouped.shape

(17, 8)

In [180]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 10

for hood in paris_grouped['Arrondissement']:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['Arrondissement'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----12th----
            venue  freq
0            Café   1.0
1     Candy Store   0.0
2  Chocolate Shop   0.0
3        Creperie   0.0
4    Cupcake Shop   0.0
5    Dessert Shop   0.0
6     Pastry Shop   0.0


----13th----
            venue  freq
0            Café  0.33
1        Creperie  0.33
2    Dessert Shop  0.33
3     Candy Store  0.00
4  Chocolate Shop  0.00
5    Cupcake Shop  0.00
6     Pastry Shop  0.00


----14th----
            venue  freq
0    Dessert Shop   1.0
1            Café   0.0
2     Candy Store   0.0
3  Chocolate Shop   0.0
4        Creperie   0.0
5    Cupcake Shop   0.0
6     Pastry Shop   0.0


----15th----
            venue  freq
0        Creperie   1.0
1            Café   0.0
2     Candy Store   0.0
3  Chocolate Shop   0.0
4    Cupcake Shop   0.0
5    Dessert Shop   0.0
6     Pastry Shop   0.0


----16th----
            venue  freq
0            Café  0.67
1    Dessert Shop  0.33
2     Candy Store  0.00
3  Chocolate Shop  0.00
4        Creperie  0.00
5    Cupcake Sh

In [0]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [197]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Arrondissement']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Arrondissement'] = paris_grouped['Arrondissement']

for ind in np.arange(paris_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Arrondissement,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,12th,Café,Pastry Shop,Dessert Shop,Cupcake Shop,Creperie
1,13th,Dessert Shop,Creperie,Café,Pastry Shop,Cupcake Shop
2,14th,Dessert Shop,Pastry Shop,Cupcake Shop,Creperie,Chocolate Shop
3,15th,Creperie,Pastry Shop,Dessert Shop,Cupcake Shop,Chocolate Shop
4,16th,Café,Dessert Shop,Pastry Shop,Cupcake Shop,Creperie
5,17th,Pastry Shop,Creperie,Dessert Shop,Cupcake Shop,Chocolate Shop
6,18th,Café,Pastry Shop,Dessert Shop,Cupcake Shop,Creperie
7,19th,Café,Pastry Shop,Dessert Shop,Cupcake Shop,Creperie
8,1st,Pastry Shop,Dessert Shop,Cupcake Shop,Creperie,Chocolate Shop
9,20th,Creperie,Chocolate Shop,Pastry Shop,Dessert Shop,Cupcake Shop


In [201]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 6

paris_grouped_clustering = paris_grouped.drop('Arrondissement', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([1, 0, 5, 4, 0, 2, 1, 1, 5, 3, 3, 0, 0, 2, 3, 5, 5], dtype=int32)

In [0]:
#neighbors

In [199]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

paris_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
paris_merged = paris_merged.join(neighborhoods_venues_sorted.set_index('Arrondissement'), on='Arrondissement_number')
paris_merged.head() 

,Arrondissement,Name,Arrondissement_number,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1st (Ier) R,Louvre,1st,48.862158,2.337036,5.0,Pastry Shop,Dessert Shop,Cupcake Shop,Creperie,Chocolate Shop
1,2nd (IIe) R,Bourse,2nd,48.867684,2.343126,3.0,Creperie,Chocolate Shop,Pastry Shop,Dessert Shop,Cupcake Shop
2,3rd (IIIe) R,Temple,3rd,48.862683,2.358685,0.0,Dessert Shop,Creperie,Chocolate Shop,Café,Pastry Shop
3,4th (IVe) R,Hôtel-de-Ville,4th,48.854156,2.356789,0.0,Creperie,Chocolate Shop,Café,Pastry Shop,Dessert Shop
4,5th (Ve) L,Panthéon,5th,48.845419,2.352582,2.0,Pastry Shop,Creperie,Dessert Shop,Cupcake Shop,Chocolate Shop


In [200]:
paris_merged.head(50)

,Arrondissement,Name,Arrondissement_number,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1st (Ier) R,Louvre,1st,48.862158,2.337036,5.0,Pastry Shop,Dessert Shop,Cupcake Shop,Creperie,Chocolate Shop
1,2nd (IIe) R,Bourse,2nd,48.867684,2.343126,3.0,Creperie,Chocolate Shop,Pastry Shop,Dessert Shop,Cupcake Shop
2,3rd (IIIe) R,Temple,3rd,48.862683,2.358685,0.0,Dessert Shop,Creperie,Chocolate Shop,Café,Pastry Shop
3,4th (IVe) R,Hôtel-de-Ville,4th,48.854156,2.356789,0.0,Creperie,Chocolate Shop,Café,Pastry Shop,Dessert Shop
4,5th (Ve) L,Panthéon,5th,48.845419,2.352582,2.0,Pastry Shop,Creperie,Dessert Shop,Cupcake Shop,Chocolate Shop
5,6th (VIe) L,Luxembourg,6th,48.849392,2.332260,3.0,Cupcake Shop,Creperie,Chocolate Shop,Pastry Shop,Dessert Shop
6,7th (VIIe) L,Palais-Bourbon,7th,48.854914,2.312860,5.0,Dessert Shop,Pastry Shop,Café,Cupcake Shop,Creperie
7,8th (VIIIe) R,Élysée,8th,48.877480,2.317650,NaN,NaN,NaN,NaN,NaN,NaN
8,9th (IXe) R,Opéra,9th,48.877070,2.337921,5.0,Dessert Shop,Candy Store,Pastry Shop,Cupcake Shop,Creperie
9,10th (Xe) R,Entrepôt,10th,48.876008,2.360445,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#as we can see Arrondissement 8,10,11 have not any dessert shop so are perfect place !!
#I drop them in oredr to visualize clusters
paris_merged.dropna(subset = ["Cluster Labels"], inplace=True)


In [206]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[48.8566969, 2.3514616], zoom_start=11)
folium.Marker([48.8566969, 2.3514616], popup='Paris').add_to(map_clusters)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['Arrondissement'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    print(cluster)
    cluster=int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

5.0
3.0
0.0
0.0
2.0
3.0
5.0
5.0
1.0
0.0
5.0
4.0
0.0
2.0
1.0
1.0
3.0


In [207]:
# CLUSTER 1

paris_merged.loc[paris_merged['Cluster Labels'] == 0, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Temple,0.0,Dessert Shop,Creperie,Chocolate Shop,Café,Pastry Shop
3,Hôtel-de-Ville,0.0,Creperie,Chocolate Shop,Café,Pastry Shop,Dessert Shop
12,Gobelins,0.0,Dessert Shop,Creperie,Café,Pastry Shop,Cupcake Shop
15,Passy,0.0,Café,Dessert Shop,Pastry Shop,Cupcake Shop,Creperie
